# Term Project AE642 SADC
## CLF-QP MPC (Receding Horizon)

### Imports

In [1]:

import casadi as ca

In [2]:

# --- Constants and parameters ---
# Inertia matrix (kg*m^2)
Ixx, Iyy, Izz = 0.7264, 0.7258, 1.1644   # from ITC1:contentReference[oaicite:6]{index=6}
I = ca.diag(ca.vertcat(Ixx, Iyy, Izz))
I_inv = ca.inv(I)

# Magnetorquer limits
m_max = 8.19  # A*m^2, max dipole magnitude on each axis:contentReference[oaicite:7]{index=7}

# Control and CLF weights
R_c = ca.diag(ca.vertcat(1e-2, 1e-2, 1e-2))  # small weight on m^2 (prefer minimal control)
W_s = 1e5   # heavy weight on slack (to enforce Lyapunov decrease)
gamma = 1.0 # Lyapunov decay rate (continuous)
dt = 0.2    # control step (s)
N = 20      # MPC horizon length (steps, i.e. 4 seconds)

# Earth magnetic dipole moment (for B-field model)
# We use a simple dipole aligned with inertial Z-axis (north). 
# Value chosen to give ~25 microTesla at LEO.
M_earth = 7.96e22    # A*m^2 (approx Earth dipole) 
mu0 = 1e-7 * 4*ca.pi # (mu0/4pi) factor for magnetic field


In [ ]:
# --- Define symbolic state and control ---
q = ca.SX.sym('q', 4)       # quaternion (q0, q1, q2, q3)
w = ca.SX.sym('w', 3)       # angular velocity (wx, wy, wz)
x = ca.vertcat(q, w)        # state vector (7x1)

m = ca.SX.sym('m', 3)       # control (magnetic dipole vector)

# Quaternion kinematics: q_dot = 0.5 * Omega(w) * q
# Define Omega(w) as a 4x4 matrix 
wx, wy, wz = w[0], w[1], w[2]
Omega = ca.vertcat(
    ca.horzcat(0, -wx, -wy, -wz),
    ca.horzcat(wx,  0,  wz, -wy),
    ca.horzcat(wy, -wz,  0,  wx),
    ca.horzcat(wz,  wy, -wx,  0)
)

q_dot = 0.5 * Omega @ q  # quaternion derivative (not normalized automatically)

# Rotational dynamics: w_dot = I^{-1} * ( m x B - w x (I w) )
# We will treat B as a *parameter* that can be updated at runtime (B in body frame).
B = ca.SX.sym('B', 3)  # Earth magnetic field in body coords (treated as known input)
# Compute cross products:
w_cross_Iw = ca.cross(w, I @ w)
m_cross_B = ca.cross(m, B)
w_dot = I_inv @ (m_cross_B - w_cross_Iw)

# Continuous-time state derivative
x_dot = ca.vertcat(q_dot, w_dot)

# --- Discrete dynamics (one step integration) ---
# We'll use simple Euler integration for 1 step (small dt=0.2s). 
# (For better accuracy, one could use ca.integrator with RK4.)
x_next = x + dt * x_dot

# Optionally, enforce quaternion normalization in the integration step:
# Normalize q_next to unit length to avoid drift.
q_next = x_next[0:4] / ca.norm_2(x_next[0:4])
x_next = ca.vertcat(q_next, x_next[4:7])
